In [1]:
import sounddevice
#from scipy.io.wavfile import write

#def voice_recorder(seconds, file):
def voice_recorder(seconds):
    print("Recording Started…")
    recording = sounddevice.rec((seconds * 22050), samplerate= 22050, channels=1)   # 44100/2=22050
    sounddevice.wait()
    #write(file, 44100, recording)
    
    return recording

    print("Recording Finished")

In [2]:
import numpy as np

frame_size = 1.0
sr = 44100/2
n_frame = 10

rec_frame_vectors = np.zeros((10, 22050))

for i in range(n_frame):
    rec_frame_vectors[i] = voice_recorder(1).reshape(1, -1)
    print(rec_frame_vectors[i].reshape(1, -1))
    
    # 여기에 모델을 불러와서 결과를 출력하는 코드 추가

Recording Started…
[[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Recording Started…
[[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Recording Started…
[[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Recording Started…
[[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Recording Started…
[[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Recording Started…
[[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Recording Started…
[[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Recording Started…
[[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [3]:
rec_frame_vectors.shape   # 1초씩 10개의 녹음 데이터가 실시간으로 기록

(10, 22050)

In [4]:
import torch
import torch.nn as nn

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=(4, 4), stride=2, padding=0),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=(4, 4), stride=2, padding=0),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 ?x?x64 inputs -> 1 outputs
        self.fc1 = torch.nn.Linear(896, 16, bias=True)       
        self.fc2 = torch.nn.Linear(16, 1, bias=True)       
        
        self.sigmoid = torch.nn.Sigmoid()     # 출력이 0 or 1이 되도록 이진 분류
        
        # 전결합층 한정으로 가중치 초기
        torch.nn.init.xavier_uniform_(self.fc1.weight)        
        torch.nn.init.zeros_(self.fc1.bias)
        torch.nn.init.xavier_uniform_(self.fc2.weight)        
        torch.nn.init.zeros_(self.fc2.bias)

        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc1(out)        
        out = self.fc2(out)
        out = self.sigmoid(out)
        #out = torch.round(out)    # 텐서 반올림 방법 -> 0 또는 1만 출력
        
        return out

In [5]:
# 모델 로딩하는 방법 
import torch

model = CNN()    

# 모델 구조를 미리 선언하고 로딩할 것 (230203)
#model = torch.load("model_bird.pth")   # 에러 발생
#print(model)

model.load_state_dict(torch.load("model_bird.pth"))
model.eval()

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=896, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [13]:
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=896, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [6]:
'''
# 모델 저장
torch.save(model.state_dict(), 'model_bird.pth')

#https://developers-shack.tistory.com/10
'''

"\n# 모델 저장\ntorch.save(model.state_dict(), 'model_bird.pth')\n\n#https://developers-shack.tistory.com/10\n"

In [7]:
import numpy as np
import librosa
import torch

frame_size = 1.0
sr = 22050
n_frame = 20

for i in range(n_frame):
    rec_frame_vectors = voice_recorder(1).reshape(1, -1)
    #print(rec_frame_vectors.reshape(1, -1))
    
    Spectrum_vector = librosa.feature.melspectrogram(y=rec_frame_vectors, sr=sr)
    Spectrum_vector = librosa.power_to_db(Spectrum_vector, ref=np.max)   # 특성 변경을 위해 추가(230203)
    
    print(Spectrum_vector.shape)
    
    # 여기에 모델을 불러와서 실시간 녹음 결과를 검토하는 코드 추가
    
    with torch.no_grad():
        
        model.eval()
        inputs = torch.FloatTensor([Spectrum_vector])    # 대괄호 추가 -> 매트릭스 형태 (1, 1, 128, 44) (230203)
        outputs = model(inputs)
        
        print(outputs.numpy()[0][0])
        
        if round(outputs.numpy()[0][0]) == 0:
            print("Bird Noise!")
            
        if round(outputs.numpy()[0][0]) == 1:
            print("No Bird Noise!")

Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…


C:\Users\106761\AppData\Local\Temp\ipykernel_16728\586313712.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  inputs = torch.FloatTensor([Spectrum_vector])    # 대괄호 추가 -> 매트릭스 형태 (1, 1, 128, 44) (230203)


(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0.89820236
No Bird Noise!
Recording Started…
(1, 128, 44)
0

In [8]:
# 랩탑 마이크로폰 문제로 결과 확인 어려움
# 학습에 사용된 음원을 다시 사용하여 실제 예측 성능을 확인 (230206)

import numpy as np
import librosa
import torch

frame_size = 1.0
sr = 22050
n_frame = 220

bird_audio, bird_sr = librosa.load("catbirds_-_lyrebirds_-_honeyeaters_-_thornbills_and_insects_TFo.wav")  # 학습에 사용된 음원을 한번 더 사용



for i in range(n_frame):
    
    rec_frame_vectors = bird_audio[i*22050:(i+1)*22050]       # 1초 단위로 프레임 절달 / 오버랩 되지 않고 등간격 절단 가정(230206)
    rec_frame_vectors = rec_frame_vectors.reshape(1, -1)
    #print(rec_frame_vectors.shape)
    
    Spectrum_vector = librosa.feature.melspectrogram(y=rec_frame_vectors, sr=sr)
    Spectrum_vector = librosa.power_to_db(Spectrum_vector, ref=np.max)   # 특성 변경을 위해 추가(230203)
    
    #print(Spectrum_vector.shape)
    
    # 여기에 모델을 불러와서 실시간 녹음 결과를 검토하는 코드 추가
    
    with torch.no_grad():
        
        model.eval()
        inputs = torch.FloatTensor([Spectrum_vector])    # 대괄호 추가 -> 매트릭스 형태 (1, 1, 128, 44) (230203)
        #print(inputs.shape)
        outputs = model(inputs)
        
        print(outputs.numpy()[0][0])
        print(i)
        
        if round(outputs.numpy()[0][0]) == 0:
            print("Bird Noise!")
            
        if round(outputs.numpy()[0][0]) == 1:
            print("")
            #print("No Bird Noise!")
            
        # 몇 번째 프레임에서 검출되는지 확인 (230206) -> 모든 프레임에 대하여 0.99 의 결과값을 예측
            

0.9998404
0

0.99676055
1

0.9985421
2

0.9970534
3

0.99701595
4

0.9990256
5

0.997526
6

0.99794334
7

0.99502665
8

0.97695553
9

0.910672
10

0.49302655
11
Bird Noise!
0.75795794
12

0.4169193
13
Bird Noise!
0.0099977385
14
Bird Noise!
0.0031849942
15
Bird Noise!
0.0022285953
16
Bird Noise!
0.008254167
17
Bird Noise!
0.957893
18

0.9962794
19

0.9969066
20

0.9984492
21

0.9972932
22

0.99729127
23

0.86850506
24

0.9977087
25

0.9836352
26

0.998425
27

0.99962866
28

0.9991517
29

0.9996556
30

0.9854027
31

0.9970632
32

0.99724233
33

0.99939823
34

0.9995358
35

0.9955205
36

0.9980089
37

0.996375
38

0.9719114
39

0.99277824
40

0.99577457
41

0.9909711
42

0.99096817
43

0.12213024
44
Bird Noise!
0.2739909
45
Bird Noise!
0.080281526
46
Bird Noise!
0.85274726
47

0.9982766
48

0.99867356
49

0.9993777
50

0.9961079
51

0.99803644
52

0.9959623
53

0.99964416
54

0.9941695
55

0.89466316
56

0.9991861
57

0.9999585
58

0.9952863
59

0.7321084
60

0.98617536
61

0.52440274
62

In [9]:
# 12, 14~19 프레임(초)
# 45~47 프레임(초)
# 64~65 프레임(초)
# 85~89 프레임(초)
# 114~115 프레임(초)
# 131 프레임(초)
# 135~141 프레임(초)
# 169~174 프레임(초)
# 191 프레임(초)
# 197~203 프레임(초)
# 216~220 프레임 (초)

# 소음 발생 구간 정확하게 판단 (230106)

### 변조 (진폭, 주파수), 시간 지연 등의 음원 파일 수정 경우, 정확도 검증 (230206)

In [12]:
# 약간의 진폭 변조, Noise resuction 실행 후에도 동일한 음원 파일에 대하여 정확한 예측
# 실제 특정 주파수 영역의 신호를 특성으로 사용하므로,
# 약간의 진폭 변조는 영향을 미치지 않는 것으로 판단됨 (230206)